# Normalized Voxels: Align Planes, Adjust Contrast, and Crop

As shown in several notebooks, MRI plane type (Axial, Coronal, and Sagittal) is not consistent among patients or MRI scan types (FLAIR, T1w, T1wCE, T2w).
While augmentations might alleviate this inconsistency, it is better to train models using MRI voxels that are consistent in terms of plane type.
This notebook shows we can obtain normalized voxels by appropriately rotating MRI voxels.
I found that simply rotating MRI voxels is not enough because the order of planes is also inconsistent in some cases.
For example, even with the same Sagittal type, some of scans were in left-to-right order, while others were the other way around.
As Instance Number does not help, Image Position (Patient) is used in this notebook to reorder stacked images.
After normalized voxels with respect to planes, contrast is adjusted and then voxels are cropped.
Finally, the voxel is resized to arbitrary fixed size.

## Normalized Voxel Datasets

The normalized voxels created the above procedure were stored as a dataset. Please refer to the second half of this notebook.

In [ ]:
from pathlib import Path
import numpy as np
import cv2
import pydicom
import matplotlib.pyplot as plt

DATASET = 'train'
scan_types = ['FLAIR','T1w','T1wCE','T2w']
data_root = Path("../input/rsna-miccai-brain-tumor-radiogenomic-classification")

In [ ]:
# https://www.kaggle.com/arnabs007/part-1-rsna-miccai-btrc-understanding-the-data
# https://www.kaggle.com/davidbroberts/determining-mr-image-planes
def get_image_plane(data):
    x1, y1, _, x2, y2, _ = [round(j) for j in data.ImageOrientationPatient]
    cords = [x1, y1, x2, y2]

    if cords == [1, 0, 0, 0]:
        return 'Coronal'
    elif cords == [1, 0, 0, 1]:
        return 'Axial'
    elif cords == [0, 1, 0, 0]:
        return 'Sagittal'
    else:
        return 'Unknown'

In [ ]:
def get_voxel(study_id, scan_type):
    imgs = []
    dcm_dir = data_root.joinpath(DATASET, study_id, scan_type)
    dcm_paths = sorted(dcm_dir.glob("*.dcm"), key=lambda x: int(x.stem.split("-")[-1]))
    positions = []
    
    for dcm_path in dcm_paths:
        img = pydicom.dcmread(str(dcm_path))
        imgs.append(img.pixel_array)
        positions.append(img.ImagePositionPatient)
        
    plane = get_image_plane(img)
    voxel = np.stack(imgs)
    
    # reorder planes if needed and rotate voxel
    if plane == "Coronal":
        if positions[0][1] < positions[-1][1]:
            voxel = voxel[::-1]
            print(f"{study_id} {scan_type} {plane} reordered")
        voxel = voxel.transpose((1, 0, 2))
    elif plane == "Sagittal":
        if positions[0][0] < positions[-1][0]:
            voxel = voxel[::-1]
            print(f"{study_id} {scan_type} {plane} reordered")
        voxel = voxel.transpose((1, 2, 0))
        voxel = np.rot90(voxel, 2, axes=(1, 2))
    elif plane == "Axial":
        if positions[0][2] > positions[-1][2]:
            voxel = voxel[::-1]
            print(f"{study_id} {scan_type} {plane} reordered")
        voxel = np.rot90(voxel, 2)
    else:
        raise ValueError(f"Unknown plane {plane}")
    return voxel, plane

In [ ]:
def normalize_contrast(voxel):
    if voxel.sum() == 0:
        return voxel
    voxel = voxel - np.min(voxel)
    voxel = voxel / np.max(voxel)
    voxel = (voxel * 255).astype(np.uint8)
    return voxel

In [ ]:
def crop_voxel(voxel):
    if voxel.sum() == 0:
        return voxel
    keep = (voxel.mean(axis=(0, 1)) > 0)
    voxel = voxel[:, :, keep]
    keep = (voxel.mean(axis=(0, 2)) > 0)
    voxel = voxel[:, keep]
    keep = (voxel.mean(axis=(1, 2)) > 0)
    voxel = voxel[keep]
    return voxel

Sample planes along the longest axis and resize the sampled planes.
By sampling along the longest axis, the degradation due to sampling is minimized.
The best way is to resize twice (e.g. (x, y) axis then (y, z) axis) but it is computationally expensive.

In [ ]:
def resize_voxel(voxel, sz=64):
    output = np.zeros((sz, sz, sz), dtype=np.uint8)

    if np.argmax(voxel.shape) == 0:
        for i, s in enumerate(np.linspace(0, voxel.shape[0] - 1, sz)):
            output[i] = cv2.resize(voxel[int(s)], (sz, sz))
    elif np.argmax(voxel.shape) == 1:
        for i, s in enumerate(np.linspace(0, voxel.shape[1] - 1, sz)):
            output[:, i] = cv2.resize(voxel[:, int(s)], (sz, sz))
    elif np.argmax(voxel.shape) == 2:
        for i, s in enumerate(np.linspace(0, voxel.shape[2] - 1, sz)):
            output[:, :, i] = cv2.resize(voxel[:, :, int(s)], (sz, sz))

    return output

In [ ]:
for study_path in list(data_root.joinpath(DATASET).glob("*"))[:5]:
    study_id = study_path.name
    
    if not study_path.is_dir():
        continue

    fig = plt.figure(figsize=(12, 9))
    fig.suptitle(f"study id: {study_id}", fontsize=24)

    for i, scan_type in enumerate(scan_types):
        voxel, plane = get_voxel(study_id, scan_type)
        voxel = normalize_contrast(voxel)
        voxel = crop_voxel(voxel)
        voxel = resize_voxel(voxel)
        ax = fig.add_subplot(3, 4, i + 1)
        ax.set_title(f"{scan_type}, {plane}")
        ax.imshow(voxel[voxel.shape[0] // 2], cmap='gray')
        ax = fig.add_subplot(3, 4, i + 1 + 4)
        ax.imshow(voxel[:, voxel.shape[1] // 2], cmap='gray')
        ax = fig.add_subplot(3, 4, i + 1 + 8)
        ax.imshow(voxel[:, :, voxel.shape[2] // 2], cmap='gray')
    
    fig.show()

# Normalized Voxel Datasets
The normalized voxels created the above procedure were stored as a dataset:

- [64x64x64 voxel](https://www.kaggle.com/ren4yu/rsna-miccai-voxel-64-dataset)
- [128x128x128 voxel](https://www.kaggle.com/ren4yu/rsna-miccai-voxel-128-dataset)
- [256x256x256 voxel](https://www.kaggle.com/ren4yu/rsna-miccai-voxel-256-dataset)

The directory structure is as follows:

```
voxel
├── train
│   ├── 00000
│   │   ├── FLAIR.npy
│   │   ├── T1w.npy
│   │   ├── T1wCE.npy
│   │   └── T2w.npy
│   ├── 00002
│   │   ├── FLAIR.npy
...
├── test
│   ├── 00001
│   │   ├── FLAIR.npy
│   │   ├── T1w.npy
│   │   ├── T1wCE.npy
│   │   └── T2w.npy
│   ├── 00013
│   │   ├── FLAIR.npy
```

Some voxels do not exist because all images belonging to these scans are completely black:

- ('train', '00109', 'FLAIR.npy')
- ('train', '00123', 'T1w.npy')
- ('train', '00123', 'T2w.npy')
- ('train', '00709', 'FLAIR.npy')

Let's get one voxel and visualize it.

In [ ]:
! ls ../input

In [ ]:
def load_voxel(study_id, scan_type="FLAIR", split="train", sz=256):
    assert sz in (64, 128, 256)
    data_root = Path(f"../input/rsna-miccai-voxel-{sz}-dataset")
    npy_path = Path(data_root).joinpath("voxel", split, study_id, f"{scan_type}.npy")
    voxel = np.load(str(npy_path))
    return voxel

In [ ]:
voxel = load_voxel("00000", "FLAIR", "train", 256)
voxel.shape

In [ ]:
fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot(1, 3, 1)
ax.imshow(voxel[voxel.shape[0] // 2], cmap='gray')
ax = fig.add_subplot(1, 3, 2)
ax.imshow(voxel[:, voxel.shape[1] // 2], cmap='gray')
ax = fig.add_subplot(1, 3, 3)
ax.imshow(voxel[:, :, voxel.shape[2] // 2], cmap='gray')
fig.show()